In [1]:
#from ipynb.fs.full.adw_sql_query_issue import get_table_schemas_of_query
#import ipynb.fs.full.adw_sql_query_issue
import import_ipynb
import adw_sql_query_issue


In [85]:
example_query1 = '''
select
  EAD.ENCNTR_ID,
  ead.beg_dt_tm at time zone 'America/New_York' AD_BEGIN_DT_TM,
  ead.end_dt_tm at time zone 'America/New_York' AD_END_DT_TM,
  (EAD.END_DT_TM - EAD.BEG_DT_TM) AD_TOTAL,
  (
    select DISPLAY
    from RRMC_P0148.CODE_VALUE
    where (
      ACTIVE_IND = 1
      and CODE_SET = 4002595
      and CODE_VALUE = EAD.REASON_TYPE_CD
    )
  ) AD_REASON_TYPE,
  (
    select DISPLAY
    from RRMC_P0148.CODE_VALUE
    where (
      ACTIVE_IND = 1
      and CODE_SET = 4002596
      and CODE_VALUE = EAD.REASON_CD
    )
  ) AD_REASON,
  P.NAME_FULL_FORMATTED AD_CARE_MANAGER_NAME
from RRMC_P0148.ENCNTR_AVOIDABLE_DAYS EAD
  join RRMC_P0148.PRSNL P
    on EAD.UPDT_ID = P.PERSON_ID
where EAD.ACTIVE_IND = 1
'''

In [210]:
# Using the custom context manager
result = []
with adw_sql_query_issue.SQLQueryFixSession() as session:
    # In the first step, we provide the query only.
    prompt_1 = "You are the ADW query expert. Here is the Oracle query for ADW: '{}'. Please only provide the potential issues including the error_code, error_message, and reason. If there are no errors, please do not discuss".format(example_query1)
    # print(prompt_1)
    result.append(session.chat_with_agent(prompt_1))

    # In the second step, we provide the input tables schemas
    input_table_schemas = adw_sql_query_issue.get_table_schemas_of_query(example_query1)
    print(input_table_schemas)
    prompt_2 = "Here are the schemas of related tables including the column name and type: '{}'. Can you find new issues based on this or confirm the potential issues you found?".format(input_table_schemas)
    result.append(session.chat_with_agent(prompt_2))

    # # In the third step, we provide the target tables schemas
    target_table_schemas = adw_sql_query_issue.run_sql_query_in_adw("SELECT COLUMN_NAME, DATA_TYPE FROM ALL_TAB_COLUMNS WHERE OWNER = '{}' AND TABLE_NAME = '{}'".format("RRMC_ANALYST", "RRMC_AVOIDABLE_DAYS_DTL"))
    prompt_3 = "The query result will be inserted into this table. The column name and types are {}. Can you find new issues based on this or confirm the potential issues you found?".format(target_table_schemas)
    result.append(session.chat_with_agent(prompt_3))

    # In the fourth step, we want the GenAI to provide the fixed query
    result.append(session.chat_with_agent("Please provide the fixed Oracle query only. Please only fix the confirmed issue."))

<Response [200]>
Create the session
<Response [200]>
{'RRMC_P0148.ENCNTR_AVOIDABLE_DAYS': [('TLC_FACILITY_ID', 'NUMBER'), ('POST_ACUTE_PROVIDER_ID', 'NUMBER'), ('ENCNTR_AVOIDABLE_DAYS_ID', 'NUMBER'), ('ENCNTR_ID', 'NUMBER'), ('BEG_DT_TM', 'TIMESTAMP(9)'), ('END_DT_TM', 'TIMESTAMP(9)'), ('REASON_TYPE_CD', 'NUMBER'), ('REASON_CD', 'NUMBER'), ('COMMENT_LONG_TEXT_ID', 'NUMBER'), ('RESP_PARTY_SELF_IND', 'NUMBER'), ('RESP_PARTY_PAYER_ORG_ID', 'NUMBER'), ('RESP_PARTY_PRSNL_ID', 'NUMBER'), ('RESP_PARTY_SERVICE_TYPE_CD', 'NUMBER'), ('RESP_PARTY_BUSINESS_ORG_ID', 'NUMBER'), ('RESP_PARTY_OTHER_CD', 'NUMBER'), ('ACTIVE_IND', 'NUMBER'), ('ACTIVE_STATUS_DT_TM', 'TIMESTAMP(9)'), ('ACTIVE_STATUS_PRSNL_ID', 'NUMBER'), ('ACTIVE_STATUS_CD', 'NUMBER'), ('UPDT_CNT', 'NUMBER'), ('UPDT_ID', 'NUMBER'), ('UPDT_TASK', 'NUMBER'), ('UPDT_APPLCTX', 'NUMBER'), ('UPDT_DT_TM', 'TIMESTAMP(9)'), ('RESP_PARTY_NURSE_UNIT_CD', 'NUMBER'), ('ROW_VERSION', 'NUMBER'), ('ODS_UPDATE_DT_TM', 'TIMESTAMP(9)')], 'RRMC_P0148.PRSNL':

In [212]:
text_result = [item.message.content.text for item in result]
print(len(text_result))
for index, item in enumerate(text_result):
    print(index)
    print(item)

4
0
**Error Code:** ORA-00932
**Error Message:** *expression is of data type INTERVAL DAY TO SECOND, which is incompatible with expected data type NUMBER*
**Reason:** The query is trying to subtract two timestamps and then calculate the difference in days. While this is supported in Vertica, ADW Oracle requires an explicit conversion using the `extract( day from (duration) )` function to get the duration in days.
1
The provided references and chat history are sufficient to answer your question. 

Based on the information, the potential issue with error code ORA-00932 and the associated error message and reason still stand. This issue arises due to the incompatibility between the data types INTERVAL DAY TO SECOND and NUMBER when subtracting timestamps and requires an explicit conversion using the `extract()` function. 

Additionally, I would like to highlight that the schema information you provided does not indicate any further potential issues with the query. However, please note that

In [214]:
citation_result = [item.message.content.citations for item in result]
for index, item in enumerate(citation_result):
    print(item)

[{
  "source_location": {
    "source_location_type": "OCI_OBJECT_STORAGE",
    "url": "https://objectstorage.us-chicago-1.oraclecloud.com/n/idlcbf4ihtx7/b/qct_error_messages/o/ORA-00932.txt"
  },
  "source_text": "{\n    'category': 'Oracle SQL Errors',\n    'entries': [\n      {\n        'error_code': 'ORA-00932',\n        'error_message': 'expression is of data type INTERVAL DAY TO SECOND, which is incompatible with expected data type NUMBER',\n        'possile_issues': [\n          {\n            'cause': [\n              'This ADW Oracle query is converted from the Vertica query. ',\n              'The query tries to convert the duration to the number type. The duration may be the difference between two timestamps. ',\n              'Vitica can directly convert the duration as a NUMERIC value in terms of days. ',\n              'ADW Oracle query cannot be directly used in calculations or comparisons expecting a numeric value without explicit conversion'\n            ],\n          

In [143]:
example_query2 = '''
select
  DCP_SHIFT_ASSIGNMENT.PERSON_ID,
  DCP_SHIFT_ASSIGNMENT.ENCNTR_ID,
  DCP_SHIFT_ASSIGNMENT.PRSNL_ID,
  PERSON_PLAN_RELTN.MEMBER_NBR,
  PERSON.NAME_LAST_KEY,
  PERSON.NAME_FIRST_KEY,
  PERSON.NAME_MIDDLE_KEY,
  case
    when (
      DCP_SHIFT_ASSIGNMENT.BEG_EFFECTIVE_DT_TM > '01-02-2100'
      or DCP_SHIFT_ASSIGNMENT.BEG_EFFECTIVE_DT_TM < '01-02-1800'
    ) then DCP_SHIFT_ASSIGNMENT.BEG_EFFECTIVE_DT_TM
    else dcp_shift_assignment.beg_effective_dt_tm at time zone 'America/New_York'
  end BEG_EFFECTIVE_DT_TM,
  case
    when (
      DCP_SHIFT_ASSIGNMENT.END_EFFECTIVE_DT_TM > '01-02-2100'
      or DCP_SHIFT_ASSIGNMENT.END_EFFECTIVE_DT_TM < '01-02-1800'
    ) then DCP_SHIFT_ASSIGNMENT.END_EFFECTIVE_DT_TM
    else dcp_shift_assignment.end_effective_dt_tm at time zone 'America/New_York'
  end END_EFFECTIVE_DT_TM,
  (
    select DISPLAY
    from RRMC_P0148.CODE_VALUE
    where CODE_VALUE = ASSIGNED_RELTN_TYPE_CD
  ) PRVDR_RELATIONSHIP
from RRMC_ANALYST.RRMC_DCP_SHIFT_ASSIGNMENT DCP_SHIFT_ASSIGNMENT
  join RRMC_P0148.PERSON
    on (
      PRSNL_ID = PERSON.PERSON_ID
      and DCP_SHIFT_ASSIGNMENT.PRSNL_ID <> 0
    )
  join RRMC_P0148.PERSON_PLAN_RELTN
    on DCP_SHIFT_ASSIGNMENT.PERSON_ID = PERSON_PLAN_RELTN.PERSON_ID
where (
  PERSON_PLAN_RELTN.PRIORITY_SEQ = 1
  and PERSON_PLAN_RELTN.END_EFFECTIVE_DT_TM > SYSDATE
)
'''

In [145]:
# Using the custom context manager
result = []
with adw_sql_query_issue.SQLQueryFixSession() as session:
    result.append(session.chat_with_agent("Here is the Oracle query for ADW: '{}'. Please only provide the potential issues. Please do not provide the information if the query does not have the error.".format(example_query2)))
    result.append(session.chat_with_agent("Here are the schemas of related tables including the column name and type: '{}'. Can you find new issues base this or confirm the potential issues you found?".format(adw_sql_query_issue.get_table_schemas_of_query(example_query2))))
    insert_table_schema = adw_sql_query_issue.run_sql_query_in_adw("SELECT COLUMN_NAME, DATA_TYPE FROM ALL_TAB_COLUMNS WHERE OWNER = '{}' AND TABLE_NAME = '{}'".format("RRMC_ANALYST", "RRMC_CARE_PROVIDERS"))
    result.append(session.chat_with_agent("The result of the query will be insert into this table. The column name and types are {}. Can you find new issues base this or confirm the potential issues you found?".format(insert_table_schema)))
    result.append(session.chat_with_agent("Please let us know the if the origin query has risk of any error. Please give us the potential error."))
    result.append(session.chat_with_agent("Please provide the fixed Oracle query only. Please only fix the confirmed issue."))

<Response [200]>
Create the session
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Delete the session
<Response [200]>


In [202]:
# Using the custom context manager
result = []
with adw_sql_query_issue.SQLQueryFixSession() as session:
    # In the first step, we provide the query only.
    prompt_1 = "You are the ADW query expert. Here is the Oracle query for ADW: '{}'. Please only provide the potential issues including the error_code, error_message, and reason. If there are no errors, please do not discuss".format(example_query2)
    # print(prompt_1)
    result.append(session.chat_with_agent(prompt_1))

    # In the second step, we provide the input tables schemas
    input_table_schemas = adw_sql_query_issue.get_table_schemas_of_query(example_query2)
    # print(input_table_schemas)
    prompt_2 = "Here are the schemas of related tables including the column name and type: '{}'. Can you find new issues based on this or confirm the potential issues you found?".format(input_table_schemas)
    result.append(session.chat_with_agent(prompt_2))

    # # In the third step, we provide the target tables schemas
    target_table_schemas = adw_sql_query_issue.run_sql_query_in_adw("SELECT COLUMN_NAME, DATA_TYPE FROM ALL_TAB_COLUMNS WHERE OWNER = '{}' AND TABLE_NAME = '{}'".format("RRMC_ANALYST", "RRMC_CARE_PROVIDERS"))
    prompt_3 = "The query result will be inserted into this table. The column name and types are {}. Can you find new issues based on this or confirm the potential issues you found?".format(target_table_schemas)
    result.append(session.chat_with_agent(prompt_3))

    # # In the fourth step, we want the GenAI to provide the fixed query
    result.append(session.chat_with_agent("Please provide the fixed Oracle query only. Please check the entire query and fix the confirmed issues."))

<Response [200]>
Create the session
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
Delete the session
<Response [200]>


In [204]:
text_result = [item.message.content.text for item in result]
print(len(text_result))
for index, item in enumerate(text_result):
    print(index)
    print(item)

4
0
I cannot provide an answer to this query.
1
The provided query has potential issues with the comparison of dates and timestamps. The error code 'ORA-01843' indicates an invalid month specification, and the resolution suggests that explicit type conversion is needed for string literals when comparing dates or timestamps. 

In the given query, there are comparisons made with the date '01-02-2100', which could be the root cause of the issue. The query should use the `TO_TIMESTAMP` function to explicitly convert the string literal to a timestamp with the correct format, as shown in the example:

```sql
SELECT EVENT_ID
FROM EVENTS
WHERE EVENT_START > TO_TIMESTAMP('01-02-2100', 'DD-MM-YYYY');
```

Therefore, the potential issue lies in the comparison of dates or timestamps, and the resolution is to use explicit type conversion with the `TO_TIMESTAMP` function and specify the correct format.
2
Based on the provided information, there seems to be a potential issue with the data types of th

In [208]:
adw_sql_query_issue.run_sql_query_in_adw(text_result[3][6:-4])

[(1670978,
  15263091,
  1278286,
  '076405332A',
  'SHALOO',
  'MAUREEN',
  'M',
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  ' '),
 (1670978,
  15263091,
  1278286,
  '9KM5VE3XU63',
  'SHALOO',
  'MAUREEN',
  'M',
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  ' '),
 (1595278,
  14775063,
  1276533,
  '5V68DU4TR20',
  'FLANDERS',
  'CONCEPCION',
  'M',
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  ' '),
 (1595278,
  14625521,
  1276533,
  '5V68DU4TR20',
  'FLANDERS',
  'CONCEPCION',
  'M',
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  ' '),
 (1595278,
  14625521,
  1276533,
  '5V68DU4TR20',
  'FLANDERS',
  'CONCEPCION',
  'M',
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  <java object 'oracle.sql.TIMESTAMPTZ'>,
  ' '),
 (1595278,
  15194528,
  2049786,
  '5V68DU4TR20',
  'CRISAFULLI',
  'LINDA',
  'X',
  <java object 'oracle.sql